In [1]:
import pandas as pd
import gc
import numpy as np
import tqdm

from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score
%matplotlib inline

Подгрузим train данные.

In [2]:
train = pd.read_csv('train.csv')

C:\Installed\Anaconda3_v531\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head()

,C_Year,EtalonPosName,R_Year,age_qty,boss_pos_name,boss_subord_1y_max_qty,boss_subord_1y_min_qty,boss_subord_in_1m_qty,boss_subord_in_1y_qty,boss_subord_out_1m_qty,...,up_day_nv_sum_3m,up_day_nv_sum_6m,vl_nv_KBP_sum_3m,vl_nv_KBP_sum_6m,vl_nv_MP_sum_3m,vl_nv_MP_sum_6m,vl_nv_sum_3m,vl_nv_sum_6m,vsp_cd,work_rate_pct
0,NaN,0.0,NaN,27.0,0.0,28.0,0.0,1.0,29.0,1.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0,1.0
1,C,0.0,C,24.0,0.0,31.0,26.0,2.0,23.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0
2,D,0.0,C,52.0,0.0,31.0,26.0,1.0,19.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.0
3,NaN,0.0,NaN,28.0,0.0,15.0,14.0,2.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1.0
4,NaN,0.0,NaN,25.0,0.0,13.0,12.0,0.0,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,4,1.0


Подгрузим test данные.

In [4]:
test = pd.read_csv('test.csv')

In [5]:
test.head()

,C_Year,EtalonPosName,R_Year,age_qty,boss_pos_name,boss_subord_1y_max_qty,boss_subord_1y_min_qty,boss_subord_in_1m_qty,boss_subord_in_1y_qty,boss_subord_out_1m_qty,...,up_day_nv_sum_3m,up_day_nv_sum_6m,vl_nv_KBP_sum_3m,vl_nv_KBP_sum_6m,vl_nv_MP_sum_3m,vl_nv_MP_sum_6m,vl_nv_sum_3m,vl_nv_sum_6m,vsp_cd,work_rate_pct
0,C,0.0,C,45.0,0.0,21.0,17.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196,1.0
1,B,0.0,B,26.0,0.0,27.0,24.0,1.0,9.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,681,1.0
2,NaN,0.0,NaN,26.0,0.0,20.0,18.0,0.0,11.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1175,1.0
3,NaN,0.0,NaN,33.0,0.0,59.0,52.0,3.0,33.0,2.0,...,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,485,1.0
4,C,0.0,C,25.0,1.0,19.0,0.0,1.0,25.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,366,1.0


В файле use_columns находятся описания фичей. ВНИМАНИЕ: не у всех фичей есть описания, обычно это числовые агрегаты каких то функций за определенный промежуток времени.

In [6]:
feature_description = pd.read_excel('columns_description.xlsx')

In [7]:
feature_description.head()

,name,description
0,C_Year,NaN
1,EtalonPosName,NaN
2,R_Year,NaN
3,age_qty,Возраст сотрудника (должно стать вычисляемым п...
4,boss_pos_name,Название должности непосредственного руководит...


Дополнительные данные - это информация по логинам клиентов. Их можно использовать на ваше усмотрение. Пересечение с основными данными (train/test) идет через поле **tabnum**.

In [8]:
logins = pd.read_csv('valid_logins.csv')

In [9]:
logins.head()

,business_dt,tabnum,work_time_day,work_begins,work_ends
0,2017-06-13,0,423,18:40:44,19:02:28
1,2018-10-23,0,30652,09:00:27,18:59:51
2,2017-06-22,0,1724,18:57:29,19:36:28
3,2017-04-29,0,1934,12:25:34,17:20:21
4,2018-06-10,0,4920,11:42:39,17:24:23


В бэйзлайне никакой обработки фичей не производится. Для датасетов train/test удаляются dt фичи(временные). Каждую колонку пытаемся привести к числовому типу, если не получается - заполняем NaN. Соответственно, модель для примера  будет обучаться без любых временных и категориальных фичей.

In [10]:
train.drop([i for i in train.columns if 'dt' in i],axis=1,inplace=True)
for i in tqdm.tqdm_notebook(train.columns):
    train[i] = pd.to_numeric(train[i],errors='coerce')
print(train.shape)


(449576, 392)


In [11]:
test.drop([i for i in test.columns if 'dt' in i],axis=1,inplace=True)
for i in tqdm.tqdm_notebook(test.columns):
    test[i] = pd.to_numeric(test[i],errors='coerce')
print(test.shape)


(24000, 391)


In [12]:
from xgboost import XGBClassifier

In [13]:
xgb = XGBClassifier(n_jobs=4,random_state=42)
X_train, y_train = train.drop('target_churn_lag1m',axis=1), train['target_churn_lag1m']

In [14]:
%%time
xgb.fit(X_train,y_train)

Wall time: 7min 1s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=4, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [15]:
prediction = xgb.predict_proba(test)[:,1]

Это решение даёт на паблике около 0.75564 ROC AUC

Пример файла решения, которое ожидает на вход платформа:

In [16]:
sample_submission = pd.DataFrame({'tabnum': test['tabnum'], 'prob': prediction})
sample_submission.to_csv('sample_submission.csv')